ref: https://zenn.dev/tirimen/articles/0c70e82df4add4

ref(code): https://kiseno-log.com/2021/02/03/%E3%82%AC%E3%82%A6%E3%82%B9%E9%81%8E%E7%A8%8B%E5%9B%9E%E5%B8%B0%E3%82%92python%E3%81%A7%E5%AE%9F%E8%A3%85%E3%81%97%E3%81%A6%E3%81%BF%E3%82%8B/

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import itertools

In [3]:
with open("p1_5S11ep=disp.txt") as f:
    text=f.readlines()
sharp_marker=[]

i_read_sharp_before=0
sharp_statements=[]
values=[]
#データをラベル情報を含む#ヘッダーの行とデータのクラスタに分離
for i in range(len(text)):
    if text[i][0]=="#":
        if i_read_sharp_before==0:
            if i!=0:
                sharp_statements.append(sharp_statement)
                values.append(value)
            sharp_statement=[]
            value=[]
        sharp_statement.append(text[i])
        i_read_sharp_before=1
    else:
        value.append(text[i])
        i_read_sharp_before=0

#データのクラスタについてさらに分離
freqs=[]
magns=[]
phass=[]
for i in range(len(values)):
    freq=[]
    magn=[]
    phas=[]
    for j in range(len(values[i])):
        val_data=values[i][j].replace("\n","").split("\t")
        freq.append(float(val_data[0]))
        magn.append(float(val_data[1]))
        phas.append(float(val_data[2]))
    freqs.append(freq)
    magns.append(magn)
    phass.append(phas)

In [4]:
height=[]
rad=[]

for j in range(len(sharp_statements)):
    value=sharp_statements[i][0][15:].split(";")
    for i in range(len(value)):
        if i == len(value)-1:
            val_label=value[i].split("=")[0]
            val=float(value[i].split("=")[1][:-2])
        else:
            val_label=value[i].split("=")[0]
            val=float(value[i].split("=")[1])
        if i==0:
            for l in range(len(freqs[j])):
                height.append(val)
        elif i==2:
            for l in range(len(freqs[j])):
                rad.append(val)

学習データを形成する。今回は周波数、膜厚、半径を用いて位相を推定するモデルを構築する。

In [5]:
freqs_flat=list(itertools.chain.from_iterable(freqs))
magns_flat=list(itertools.chain.from_iterable(magns))
phass_flat=list(itertools.chain.from_iterable(phass))

len(freqs_flat),len(magns_flat),len(phass_flat),len(height),len(rad)

(10792, 10792, 10792, 10792, 10792)

10792個のデータがあるので、個の中の8割を学習に用いて2割をテストに用いる。

In [6]:
import random

In [7]:
num_data=len(freqs_flat)
num_train=int(len(freqs_flat)*0.8)

index_list=random.sample(range(num_data-1), k=num_train)

In [8]:
freq_train=[]
magn_train=[]
phas_train=[]
heig_train=[]
radi_train=[]

freq_test=[]
magn_test=[]
phas_test=[]
heig_test=[]
radi_test=[]

for i in range(num_data):
    if i in index_list:
        freq_train.append(freqs_flat[i])
        magn_train.append(magns_flat[i])
        phas_train.append(phass_flat[i])
        heig_train.append(height[i])
        radi_train.append(rad[i])
    else:
        freq_test.append(freqs_flat[i])
        magn_test.append(magns_flat[i])
        phas_test.append(phass_flat[i])
        heig_test.append(height[i])
        radi_test.append(rad[i])


In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

class kernel:
    def __init__(self, s1=1.0, s2=1.0, s3=0.1):
        self.s1, self.s2, self.s3 = s1,s2,s3

    def rbf_white(self,x, x_p,i=1):
        """カーネル計算"""
        # cdist:n-ベクトル間のユークリッド距離
        if i ==1:
            k =  self.s1 * np.exp(-1.0 * cdist(x, x_p, 'sqeuclidean')/self.s2) + (self.s3 * np.eye(x.shape[0]))
        else:
            k =  self.s1 * np.exp(-1.0 * cdist(x, x_p, 'sqeuclidean')/self.s2) 
        return k
    


# サンプルデータの作成
x_train  = [freq_train,heig_train,radi_train] # 学習データ
x_test  = [freq_test,heig_test,radi_test] # テストデータ

y_train = np.array(phas_train)

In [10]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import ConstantKernel, WhiteKernel

regr = GaussianProcessRegressor(
    kernel=ConstantKernel()* RBF() + WhiteKernel(),  # カーネルの演算については後述
    alpha=0.,  # 雑音項はカーネルの中で表現しているので 0. を指定
    # optimizer=None,
)

In [11]:
len(x_train[0]),len(y_train)

(8633, 8633)

In [12]:
x_train=np.array(x_train[0][:4])
x_train.resize(num_train,1)
y_train=np.array(y_train[:4])
y_train.resize(num_train,1)

In [13]:
regr.fit(x_train, y_train)
regr.kernel_  # 推定されたカーネルと誤差項のパラメータを確認

: 

In [ ]:
y_pred, y_std = regr.predict(x_test, return_std=True) 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.gaussian_process as gp
%matplotlib inline
plt.rcParams['font.size']=15

def plt_legend_out(frameon=True):
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, frameon=frameon)


In [ ]:
# トレーニングデータの生成
n, sd = 100, 1    # 設定
theta = 1            #  回帰係数
X = np.random.normal(scale=3, size=n).reshape(n,-1)
Yob = np.dot(X,theta) + np.random.normal(scale=sd,size=n).reshape(n,-1)

# プロット
plt.figure(figsize=(12,3))
plt.subplots_adjust(wspace=0.1)

plt.subplot(131)
kk = gp.kernels.RBF()
#kk = gp.kernels.WhiteKernel()
#kk = gp.kernels.RBF() + gp.kernels.WhiteKernel()
gpm = gp.GaussianProcessRegressor(kernel=kk)
gpm.fit(X, Yob)                                # データへのフィッティング
newx = np.linspace(-7,7,100).reshape(100,-1)   # 予測点
GPf, GPv = gpm.predict(newx, return_cov=True)  # 予測値と分散共分散行列
GPsd = np.sqrt(np.diag(GPv))                   # 予測値の標準偏差
plt.ylim(-10,10)
plt.scatter(X,Yob)
plt.plot(newx, GPf,'k-', lw=2)
plt.plot(newx, GPf+GPsd,'k--', lw=1)
plt.plot(newx, GPf-GPsd,'k--', lw=1)
plt.xlabel('x')
plt.ylabel('y')
plt.title('RBF')

plt.subplot(132)
#kk = gp.kernels.RBF()
kk = gp.kernels.WhiteKernel()
#kk = gp.kernels.RBF() + gp.kernels.WhiteKernel()
gpm = gp.GaussianProcessRegressor(kernel=kk)
gpm.fit(X, Yob)                                # データへのフィッティング
newx = np.linspace(-7,7,100).reshape(100,-1)   # 予測点
GPf, GPv = gpm.predict(newx, return_cov=True)  # 予測値と分散共分散行列
GPsd = np.sqrt(np.diag(GPv))                   # 予測値の標準偏差
plt.ylim(-10,10)
plt.scatter(X,Yob)
plt.plot(newx, GPf,'k-', lw=2)
plt.plot(newx, GPf+GPsd,'k--', lw=1)
plt.plot(newx, GPf-GPsd,'k--', lw=1)
plt.xlabel('x')
plt.tick_params(left=False,labelleft=False)
plt.title('WhiteKernel')

plt.subplot(133)
#kk = gp.kernels.RBF()
#kk = gp.kernels.WhiteKernel()
kk = gp.kernels.RBF() + gp.kernels.WhiteKernel()
gpm = gp.GaussianProcessRegressor(kernel=kk)
gpm.fit(X, Yob)                                # データへのフィッティング
newx = np.linspace(-7,7,100).reshape(100,-1)   # 予測点
GPf, GPv = gpm.predict(newx, return_cov=True)  # 予測値と分散共分散行列
GPsd = np.sqrt(np.diag(GPv))                   # 予測値の標準偏差
plt.ylim(-10,10)
plt.scatter(X,Yob)
plt.plot(newx, GPf,'k-', lw=2)
plt.plot(newx, GPf+GPsd,'k--', lw=1)
plt.plot(newx, GPf-GPsd,'k--', lw=1)
plt.xlabel('x')
plt.tick_params(left=False,labelleft=False)
plt.title('RBF + WhiteKernel')

plt.show()
